# Assignment 20
--------------------------------------------------------------------------------

### In this assignment we are going to use ResNet50 pre trained model to train CIFAR100 dataset using transfer learning.
### To achieve this first we are going to download Resnet50 model for imagenet data and we are going to set trainable layes as flase for all the layers and then we are going to add few more dense layer to it to train.

### Let's first import all the required libraries


In [0]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import UpSampling2D,Flatten,BatchNormalization,Dense,Dropout
from tensorflow.keras import optimizers
import tensorflow as tf
from keras.utils import np_utils
from keras.models import load_model
from keras.datasets import cifar100
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import cv2

### Now, let's download pre trained Resnet50 model from keras for imagenet dataset

In [0]:
conv_base = ResNet50(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

In [0]:
conv_base.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 206, 206, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 100, 100, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 100, 100, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

### Now we are going to set trainable parameter as false for all layer

In [0]:
conv_base.trainable = False
for layer in conv_base.layers:
    layer.trainable = False

### Now, let's see the last few layers trainable status

In [0]:
layers = [(layer, layer.name, layer.trainable) for layer in conv_base.layers]
trainable = pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable']) 
trainable.tail(40)

,Layer Type,Layer Name,Layer Trainable
135,<tensorflow.python.keras.layers.core.Activatio...,conv4_block6_1_relu,False
136,<tensorflow.python.keras.layers.convolutional....,conv4_block6_2_conv,False
137,<tensorflow.python.keras.layers.normalization....,conv4_block6_2_bn,False
138,<tensorflow.python.keras.layers.core.Activatio...,conv4_block6_2_relu,False
139,<tensorflow.python.keras.layers.convolutional....,conv4_block6_3_conv,False
140,<tensorflow.python.keras.layers.normalization....,conv4_block6_3_bn,False
141,<tensorflow.python.keras.layers.merge.Add obje...,conv4_block6_add,False
142,<tensorflow.python.keras.layers.core.Activatio...,conv4_block6_out,False
143,<tensorflow.python.keras.layers.convolutional....,conv5_block1_1_conv,False
144,<tensorflow.python.keras.layers.normalization....,conv5_block1_1_bn,False


### Now let's downlaod CIFAR100 train and validation data set and scale  the data

In [0]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0

y_train = np_utils.to_categorical(y_train, 100)
y_test = np_utils.to_categorical(y_test, 100)

print(x_train.shape)
print(x_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


### Now let's add some more dense layer to existing resnet50 layer and compile the model

In [0]:
model = Sequential()
model.add(UpSampling2D((2,2)))
model.add(UpSampling2D((2,2)))
model.add(UpSampling2D((2,2)))
model.add(conv_base)
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(256, activation='sigmoid'))
model.add(Dense(100, activation='softmax'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-5), loss='categorical_crossentropy', metrics=['acc'])


### Now let's run the model for 8 epochs using batch size as 40

In [0]:
history = model.fit(x_train, y_train, epochs=8, batch_size=40, validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/8
50000/50000 [==============================] - 316s 6ms/sample - loss: 2.9689 - acc: 0.5230 - val_loss: 2.2828 - val_acc: 0.6317
Epoch 2/8
50000/50000 [==============================] - 310s 6ms/sample - loss: 2.0752 - acc: 0.7271 - val_loss: 1.8878 - val_acc: 0.6638
Epoch 3/8
50000/50000 [==============================] - 310s 6ms/sample - loss: 1.5688 - acc: 0.8041 - val_loss: 1.6354 - val_acc: 0.6873
Epoch 4/8
50000/50000 [==============================] - 309s 6ms/sample - loss: 1.1712 - acc: 0.8675 - val_loss: 1.4753 - val_acc: 0.6965
Epoch 5/8
50000/50000 [==============================] - 310s 6ms/sample - loss: 0.8489 - acc: 0.9204 - val_loss: 1.3699 - val_acc: 0.7043
Epoch 6/8
50000/50000 [==============================] - 310s 6ms/sample - loss: 0.5910 - acc: 0.9570 - val_loss: 1.3119 - val_acc: 0.7059
Epoch 7/8
50000/50000 [==============================] - 309s 6ms/sample - loss: 0.3958 - acc: 0.9785 - val_loss: 1.

### From the above we can see that our max validation accuracy reached 70.59% for CIFAR100 dataset using transfer learning.

In [0]:
# from keras.preprocessing.image import ImageDataGenerator

# datagen_train = ImageDataGenerator(zoom_range=0.3, rotation_range=50,
#                                    width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, 
#                                    horizontal_flip=True,featurewise_center=True, featurewise_std_normalization=True)

# datagen_test = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
# iterator_test = datagen_test.flow(x_test, y_test, batch_size=len(x_test), shuffle=False)
# x_test_2, y_test_2 = iterator_test.next()

# # train the model
# #start = time.time()
# # Train the model
# model_info = model.fit_generator(datagen_train.flow(x_train, y_train, batch_size = 128, shuffle=True), epochs = 5, validation_data = (x_test_2, y_test_2))